# Chat with my resume using Retrieval Augmented Generation
I am attempting to recreate whatever I have learnt over the day from various tutorials regarding RAG and chat bots.

In [1]:
pip install -U "transformers==4.38.1" --upgrade

installed

That obviously has to start with the aptly titled library, Transformers.

In [2]:
!pip install -q pypdf # For reading the pdf version of my resume.
!pip install -q python-dotenv # To read environment variables that you would have to search for in this repo. Quest points include free access to an open sourced language model.
!pip install  llama-index==0.10.12 # How can one create a chat interfave and not have the blessings of our llama god.
!pip install -q gradio # For the ui. Albeit I am not sure if I will be using this entirely.
!pip install einops # Let's see how I will manipulate them tensors.
!pip install accelerate # Hugging face hugging me with warmth I crave.

installed

I think that many libraries would be enough for Sam Altman to write the entire gpt model again. But not me. I need MOREEE

In [3]:
!pip install llama-index-llms-huggingface

installed

In [4]:
!pip install fastembed

installed

### OK. enough libraries Sid.

In [1]:
import logging
import sys
import os
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
documents = SimpleDirectoryReader(os.path.join(os.getcwd(),'data')).load_data()

In [2]:
# !pip install llama-index-embeddings-fastembed
from llama_index.embeddings.fastembed import FastEmbedEmbedding

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5") # The good person that wrote the tutorial reminded us that OpenAI's chatGPT embedder is the default way to go and we are rascals. We are gonna change the status quo. One day.
Settings.embed_model = embed_model
Settings.chunk_size = 512

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

ort_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [3]:
from llama_index.core import PromptTemplate


system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."


# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")



In [34]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
!huggingface-cli login --token XYZ

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/siddhantshah/.cache/huggingface/token
Login successful


In [5]:
import torch

llm = HuggingFaceLLM(
    context_window=8192,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="google/gemma-7b-it",
    model_name="google/gemma-7b-it",
    device_map="auto",
    #stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

Settings.llm = llm
Settings.chunk_size = 512

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the disk.


In [ ]:
documents

In [6]:
index = VectorStoreIndex.from_documents(documents)

In [7]:
query_engine = index.as_query_engine()

def predict(input, history):
  response = query_engine.query(input)
  return str(response)


In [8]:

import gradio as gr

gr.ChatInterface(predict).launch(share=True)

INFO:httpx:HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
HTTP Request: GET https://api.gradio.app/gradio-messaging/en "HTTP/1.1 200 OK"
Running on local URL:  http://127.0.0.1:7860
INFO:httpx:HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
HTTP Request: GET http://127.0.0.1:7860/startup-events "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
HTTP Request: HEAD http://127.0.0.1:7860/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
HTTP Request: GET https://checkip.amazonaws.com/ "HTTP/1.1 200 "
INFO:httpx:HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.gradio.app/gradio-initiated-analytics/ "HTTP/1.1 200 OK"
HTTP Request: POST https://api.gradio.app/gradio-initiated-analytics/ "HTTP/1.1 200 OK"
INFO:httpx:HTTP R

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running on public URL: https://f34b54e4c38d7d592e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
INFO:httpx:HTTP Request: HEAD https://f34b54e4c38d7d592e.gradio.live "HTTP/1.1 200 OK"
HTTP Request: HEAD https://f34b54e4c38d7d592e.gradio.live "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.gradio.app/gradio-launched-telemetry/ "HTTP/1.1 200 OK"
HTTP Request: POST https://api.gradio.app/gradio-launched-telemetry/ "HTTP/1.1 200 OK"


/opt/anaconda3/envs/chat_with_resume/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [30]:
predict('what school did he go to?','')

RuntimeError: MPS backend out of memory (MPS allocated: 16.57 GB, other allocations: 1.47 GB, max allowed: 18.13 GB). Tried to allocate 128.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).